# Adaptive robust lot sizing

In [1]:
import cvxpy as cp
import numpy as np
import torch
import matplotlib.pyplot as plt

import lropt
from lropt.parameter import Parameter
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

In [2]:
N = 30
c = 20
K = 20
dmax = 20
Gamma = 20*np.sqrt(N)
coordinates = 10*np.random.rand(2, N)
t = ((coordinates[[0]] - coordinates[[0]].T) ** 2
     + (coordinates[[1]] - coordinates[[1]].T) ** 2) ** 0.5

In [3]:
uncertainty_set = lropt.Ellipsoidal(p=1, rho=Gamma, lb=0, ub=dmax)
x = cp.Variable(N)
d = UncertainParameter(N, uncertainty_set=uncertainty_set)
y_0 = cp.Variable((N,N))
y_d = [cp.Variable((N,N)) for _ in range(N)]
y = 0
for k in range(N):
    y += y_d[k]*d[k]
y += y_0

#cp.multiply is elementwise multiplication
objective = cp.Minimize(cp.sum(c*x) + cp.sum(cp.multiply(t,y)))
constraints = [
                d <= cp.sum(y, axis=0) - cp.sum(y, axis=1) + x,
                y >= 0,
                x >= 0,
                x <= K,
              ]
prob = RobustProblem(objective=objective, constraints=constraints)

In [8]:
y.parameters()

[UncertainParameter((30,))]

In [4]:
prob.solve()

ValueError: DRP error: not able to process non multiplication/additions